In [2]:
import io
import os
import pathlib
import requests
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Downloading the data

In [3]:
ROOT_DIR = pathlib.Path("..")

DATASETS_URL = "https://github.com/KAUST-Academy/python-for-data-analysis/raw/november-2022/datasets.zip"

response = requests.get(DATASETS_URL)
z = zipfile.ZipFile(io.BytesIO(response.content))
z.extractall(ROOT_DIR)


In [4]:
DATASETS_DIR = ROOT_DIR / "datasets"
DATASET_DIR = DATASETS_DIR / "movielens"

In [5]:
!ls -F $DATASETS_DIR

babynames/     fec/    movielens/  titanic/
bitly_usagov/  haiti/  mta_perf/   usda_food/


In [6]:
!ls $DATASET_DIR

movies.dat  ratings.dat  README  users.dat


In [7]:
!cat $DATASET_DIR/README

SUMMARY

These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 
made by 6,040 MovieLens users who joined MovieLens in 2000.

USAGE LICENSE

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under the following conditions:

     * The user may not state or imply any endorsement from the
       University of Minnesota or the GroupLens Research Group.

     * The user must acknowledge the use of the data set in
       publications resulting from the use of the data set, and must
       send us an electronic or paper copy of those publications.

     * The user may not redistribute the data without separate
       permission.

     * The user may not use this information for any commercial or
       revenue-bearing purposes without first obtaining 

## Loading the data

In [8]:
!cat $DATASET_DIR/users.dat | head -n 5

1::F::1::10::48067
2::M::56::16::70072
3::M::25::15::55117
4::M::45::7::02460
5::M::25::20::55455


In [9]:
pd.read_table?

In [10]:
unames = [
    "user_id",
    "gender",
    "age",
    "occupation",
    "zip"
]

users = pd.read_table(
    DATASET_DIR / "users.dat",
    sep="::",
    header=None,
    names=unames,
    engine="python"
)


In [11]:
users.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [12]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     6040 non-null   int64 
 1   gender      6040 non-null   object
 2   age         6040 non-null   int64 
 3   occupation  6040 non-null   int64 
 4   zip         6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB


In [13]:
!cat $DATASET_DIR/ratings.dat | head -n 10

1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968
1::3408::4::978300275
1::2355::5::978824291
1::1197::3::978302268
1::1287::5::978302039
1::2804::5::978300719
1::594::4::978302268
1::919::4::978301368


In [14]:
rnames = [
    "user_id",
    "movie_id",
    "rating",
    "timestamp"
]

def from_unix_time(ts):
    return pd.to_datetime(ts, unit='s')

ratings = pd.read_table(
    DATASET_DIR / "ratings.dat",
    sep="::",
    header=None,
    names=rnames,
    engine="python",
    parse_dates=["timestamp"],
    #date_parser=lambda ts: pd.to_datetime(ts, unit='s'),
    date_parser=from_unix_time 
)


In [15]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


In [16]:
(ratings.loc[:, ["rating"]]
        .describe())

,rating
count,1.000209e+06
mean,3.581564e+00
std,1.117102e+00
min,1.000000e+00
25%,3.000000e+00
50%,4.000000e+00
75%,4.000000e+00
max,5.000000e+00


In [17]:
(ratings.loc[:, "rating"]
        .value_counts())

4    348971
3    261197
5    226310
2    107557
1     56174
Name: rating, dtype: int64

In [18]:
# Question: what is the average rating given to movies for each user?

In [19]:
user_id = 1
(ratings.loc[ratings.loc[:, "user_id"] == user_id, "rating"]
        .mean())

4.188679245283019

In [20]:
(ratings.loc[:, ["user_id", "rating"]]
        .groupby("user_id")
        .mean())

,rating
user_id,
1,4.188679
2,3.713178
3,3.901961
4,4.190476
5,3.146465
...,...
6036,3.302928
6037,3.717822
6038,3.800000


In [21]:
# Q: What is the average rating (across all users!) for each movie? 

In [22]:
movie_id = 7
ratings.loc[ratings.loc[:, "movie_id"] == movie_id, :]

,user_id,movie_id,rating,timestamp
842,10,7,4,2000-12-31 01:56:03
3409,26,7,4,2000-12-30 02:03:03
4670,34,7,3,2000-12-29 15:41:30
4833,35,7,2,2000-12-29 15:35:22
5053,36,7,4,2002-02-03 03:41:37
...,...,...,...,...
992216,5996,7,5,2000-05-02 19:52:21
993050,6000,7,4,2000-04-28 00:49:07
995896,6016,7,3,2000-04-26 21:41:39
997545,6025,7,4,2000-04-26 06:40:15


In [23]:
(ratings.loc[ratings.loc[:, "movie_id"] == movie_id, "rating"]
        .mean())

3.410480349344978

In [24]:
(ratings.loc[:, ["movie_id", "rating"]]
        .groupby("movie_id")
        .mean())

,rating
movie_id,
1,4.146846
2,3.201141
3,3.016736
4,2.729412
5,3.006757
...,...
3948,3.635731
3949,4.115132
3950,3.666667


In [25]:
!cat $DATASET_DIR/movies.dat | head -n 5

1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy


In [26]:
mnames = [
    "movie_id",
    "title",
    "genres"
]
movies = pd.read_table(
    DATASET_DIR / "movies.dat",
    sep="::",
    header=None,
    names=mnames,
    engine="python"
)

In [27]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  3883 non-null   int64 
 1   title     3883 non-null   object
 2   genres    3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [28]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## Combining the data

In [29]:
pd.merge?

In [31]:
users.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [32]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


In [30]:
(pd.merge(users, ratings)
   .head())

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp
0,1,F,1,10,48067,1193,5,2000-12-31 22:12:40
1,1,F,1,10,48067,661,3,2000-12-31 22:35:09
2,1,F,1,10,48067,914,3,2000-12-31 22:32:48
3,1,F,1,10,48067,3408,4,2000-12-31 22:04:35
4,1,F,1,10,48067,2355,5,2001-01-06 23:38:11


In [33]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


In [34]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [35]:
(pd.merge(ratings, movies)
   .head())

,user_id,movie_id,rating,timestamp,title,genres
0,1,1193,5,2000-12-31 22:12:40,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,2000-12-31 21:33:33,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,2000-12-30 23:49:39,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,2000-12-30 18:01:19,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,2000-12-30 06:41:11,One Flew Over the Cuckoo's Nest (1975),Drama


In [36]:
users_and_ratings = pd.merge(users, ratings)
data = pd.merge(users_and_ratings, movies)

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   user_id     1000209 non-null  int64         
 1   gender      1000209 non-null  object        
 2   age         1000209 non-null  int64         
 3   occupation  1000209 non-null  int64         
 4   zip         1000209 non-null  object        
 5   movie_id    1000209 non-null  int64         
 6   rating      1000209 non-null  int64         
 7   timestamp   1000209 non-null  datetime64[ns]
 8   title       1000209 non-null  object        
 9   genres      1000209 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 83.9+ MB


In [38]:
data.head()

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres
0,1,F,1,10,48067,1193,5,2000-12-31 22:12:40,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,2000-12-31 21:33:33,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,2000-12-30 23:49:39,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,2000-12-30 18:01:19,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,2000-12-30 06:41:11,One Flew Over the Cuckoo's Nest (1975),Drama


## Exploring the data

### Average ratings by gender

In [40]:
data.pivot_table?

In [41]:
mean_ratings = data.pivot_table(
    "rating",
    index="title",
    columns="gender",
    aggfunc="mean"
)

In [42]:
mean_ratings.head(5)

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024


### Considering only movies with sufficient ratings

In [43]:
data.head()

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres
0,1,F,1,10,48067,1193,5,2000-12-31 22:12:40,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,2000-12-31 21:33:33,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,2000-12-30 23:49:39,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,2000-12-30 18:01:19,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,2000-12-30 06:41:11,One Flew Over the Cuckoo's Nest (1975),Drama


In [45]:
(data.groupby("title")
     .size())

title
$1,000,000 Duck (1971)                         37
'Night Mother (1986)                           70
'Til There Was You (1997)                      52
'burbs, The (1989)                            303
...And Justice for All (1979)                 199
                                             ... 
Zed & Two Noughts, A (1985)                    29
Zero Effect (1998)                            301
Zero Kelvin (Kjærlighetens kjøtere) (1995)      2
Zeus and Roxanne (1997)                        23
eXistenZ (1999)                               410
Length: 3706, dtype: int64

In [46]:
ratings_by_title = data.groupby("title").size()

In [47]:
ratings_by_title.head()

title
$1,000,000 Duck (1971)            37
'Night Mother (1986)              70
'Til There Was You (1997)         52
'burbs, The (1989)               303
...And Justice for All (1979)    199
dtype: int64

In [48]:
minimum_ratings_threshold = 250
ratings_by_title >= minimum_ratings_threshold

title
$1,000,000 Duck (1971)                        False
'Night Mother (1986)                          False
'Til There Was You (1997)                     False
'burbs, The (1989)                             True
...And Justice for All (1979)                 False
                                              ...  
Zed & Two Noughts, A (1985)                   False
Zero Effect (1998)                             True
Zero Kelvin (Kjærlighetens kjøtere) (1995)    False
Zeus and Roxanne (1997)                       False
eXistenZ (1999)                                True
Length: 3706, dtype: bool

In [50]:
active_titles = ratings_by_title.index[ratings_by_title >= minimum_ratings_threshold]

In [51]:
mean_ratings = mean_ratings.loc[active_titles]

In [52]:
mean_ratings.head()

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421


### Top rated movies by gender

In [53]:
top_female_ratings = mean_ratings.sort_values("F", ascending=False)

In [54]:
top_female_ratings.head()

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415


In [55]:
top_male_ratings = mean_ratings.sort_values("M", ascending=False)

In [56]:
top_male_ratings.head()

gender,F,M
title,,
"Godfather, The (1972)",4.314700,4.583333
Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954),4.481132,4.576628
"Shawshank Redemption, The (1994)",4.539075,4.560625
Raiders of the Lost Ark (1981),4.332168,4.520597
"Usual Suspects, The (1995)",4.513317,4.518248


### Movies with largest gender diff

In [57]:
mean_ratings["diff"] = mean_ratings["M"] - mean_ratings["F"]

In [58]:
mean_ratings.head()

gender,F,M,diff
title,,,
"'burbs, The (1989)",2.793478,2.962085,0.168607
10 Things I Hate About You (1999),3.646552,3.311966,-0.334586
101 Dalmatians (1961),3.791444,3.500000,-0.291444
101 Dalmatians (1996),3.240000,2.911215,-0.328785
12 Angry Men (1957),4.184397,4.328421,0.144024


In [60]:
sorted_by_diff = mean_ratings.sort_values("diff", ascending=True)
sorted_by_diff.head()

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777


In [61]:
sorted_by_diff.tail()

gender,F,M,diff
title,,,
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
"Longest Day, The (1962)",3.411765,4.031447,0.619682
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351


### Most divisive movies

In [70]:
(data.groupby("title")
     .mean()
     .sort_values("rating", ascending=False))

,user_id,age,occupation,movie_id,rating
title,,,,,
Ulysses (Ulisse) (1954),3485.0,25.0,0.0,3172.0,5.0
Lured (1947),5313.0,56.0,0.0,3656.0,5.0
Follow the Bitch (1998),2869.0,50.0,17.0,1830.0,5.0
Bittersweet Motel (2000),2885.0,18.0,1.0,3881.0,5.0
Song of Freedom (1936),5334.0,56.0,13.0,3382.0,5.0
...,...,...,...,...,...
"Fantastic Night, The (La Nuit Fantastique) (1949)",3610.0,18.0,6.0,3376.0,1.0
Cheetah (1989),2106.0,18.0,20.0,2039.0,1.0
Torso (Corpi Presentano Tracce di Violenza Carnale) (1973),3022.5,9.5,7.0,3493.0,1.0


In [73]:
mean_ratings.head()

gender,F,M,diff
title,,,
"'burbs, The (1989)",2.793478,2.962085,0.168607
10 Things I Hate About You (1999),3.646552,3.311966,-0.334586
101 Dalmatians (1961),3.791444,3.500000,-0.291444
101 Dalmatians (1996),3.240000,2.911215,-0.328785
12 Angry Men (1957),4.184397,4.328421,0.144024


In [74]:
mean_ratings["squared_diff"] = mean_ratings["diff"]**2

In [76]:
mean_ratings.sort_values("squared_diff", ascending=True)

gender,F,M,diff,squared_diff
title,,,,
Tender Mercies (1983),3.905405,3.905263,-0.000142,2.023435e-08
"Mystery, Alaska (1999)",3.434783,3.435780,0.000997,9.944234e-07
Jerry Maguire (1996),3.758315,3.759424,0.001109,1.229099e-06
Trainspotting (1996),3.958974,3.960432,0.001457,2.123711e-06
Robin Hood: Prince of Thieves (1991),3.161290,3.163347,0.002056,4.228333e-06
...,...,...,...,...
Dumb & Dumber (1994),2.697987,3.336595,0.638608,4.078206e-01
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359,4.574612e-01
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359,4.574617e-01


In [81]:
good_movies = (mean_ratings.loc[:, 'M'] > 4) & (mean_ratings.loc[:, 'F'] > 4)

In [85]:
(mean_ratings.loc[good_movies, :]
             .sort_values("squared_diff", ascending=True))

gender,F,M,diff,squared_diff
title,,,,
Good Will Hunting (1997),4.174672,4.177064,0.002392,0.000006
Cat on a Hot Tin Roof (1958),4.046296,4.049080,0.002783,0.000008
"Fugitive, The (1993)",4.100457,4.104046,0.003590,0.000013
"Usual Suspects, The (1995)",4.513317,4.518248,0.004931,0.000024
Roger & Me (1989),4.077778,4.072816,-0.004962,0.000025
...,...,...,...,...
Braveheart (1995),4.016484,4.297839,0.281355,0.079161
Saving Private Ryan (1998),4.114783,4.398941,0.284159,0.080746
"Treasure of the Sierra Madre, The (1948)",4.044944,4.348901,0.303957,0.092390


In [88]:
movies.loc[movies.loc[:, "title"] == "Braveheart (1995)", :]

,movie_id,title,genres
108,110,Braveheart (1995),Action|Drama|War


In [89]:
data.head()

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres
0,1,F,1,10,48067,1193,5,2000-12-31 22:12:40,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,2000-12-31 21:33:33,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,2000-12-30 23:49:39,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,2000-12-30 18:01:19,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,2000-12-30 06:41:11,One Flew Over the Cuckoo's Nest (1975),Drama


In [91]:
(data.loc[:, ["title", "rating"]]
     .groupby("title")
     .std()
     .sort_values("rating", ascending=False))

,rating
title,
Foreign Student (1994),2.828427
Criminal Lovers (Les Amants Criminels) (1999),2.309401
Identification of a Woman (Identificazione di una donna) (1982),2.121320
Tokyo Fist (1995),2.121320
Better Living (1998),2.121320
...,...
Wend Kuuni (God's Gift) (1982),NaN
White Boys (1999),NaN
Windows (1980),NaN


In [92]:
rating_std_by_title = data.groupby("title")["rating"].std()
rating_std_by_title = rating_std_by_title.loc[active_titles]

In [93]:
rating_std_by_title.head()

title
'burbs, The (1989)                   1.107760
10 Things I Hate About You (1999)    0.989815
101 Dalmatians (1961)                0.982103
101 Dalmatians (1996)                1.098717
12 Angry Men (1957)                  0.812731
Name: rating, dtype: float64

In [94]:
(rating_std_by_title.sort_values(ascending=False)
                    .head(n=10))

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

### Movie ratings by Genre

In [96]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [99]:
(movies.loc[:, "genres"]
       .str
       .split("|"))

0        [Animation, Children's, Comedy]
1       [Adventure, Children's, Fantasy]
2                      [Comedy, Romance]
3                        [Comedy, Drama]
4                               [Comedy]
                      ...               
3878                            [Comedy]
3879                             [Drama]
3880                             [Drama]
3881                             [Drama]
3882                   [Drama, Thriller]
Name: genres, Length: 3883, dtype: object

In [100]:
movies.pop?

In [101]:
movies["genres"] = (movies.pop("genres")
                          .str
                          .split("|"))

In [102]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),"[Animation, Children's, Comedy]"
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [104]:
movies_exploded = movies.explode("genres")

In [105]:
movies_exploded.head(n=10)

,movie_id,title,genres
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children's
0,1,Toy Story (1995),Comedy
1,2,Jumanji (1995),Adventure
1,2,Jumanji (1995),Children's
1,2,Jumanji (1995),Fantasy
2,3,Grumpier Old Men (1995),Comedy
2,3,Grumpier Old Men (1995),Romance
3,4,Waiting to Exhale (1995),Comedy
3,4,Waiting to Exhale (1995),Drama


In [106]:
ratings_with_genre = pd.merge(pd.merge(movies_exploded, ratings), users)

In [108]:
ratings_with_genre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2101815 entries, 0 to 2101814
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   movie_id    int64         
 1   title       object        
 2   genres      object        
 3   user_id     int64         
 4   rating      int64         
 5   timestamp   datetime64[ns]
 6   gender      object        
 7   age         int64         
 8   occupation  int64         
 9   zip         object        
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 176.4+ MB


In [109]:
ratings_with_genre.head()

,movie_id,title,genres,user_id,rating,timestamp,gender,age,occupation,zip
0,1,Toy Story (1995),Animation,1,5,2001-01-06 23:37:48,F,1,10,48067
1,1,Toy Story (1995),Children's,1,5,2001-01-06 23:37:48,F,1,10,48067
2,1,Toy Story (1995),Comedy,1,5,2001-01-06 23:37:48,F,1,10,48067
3,48,Pocahontas (1995),Animation,1,5,2001-01-06 23:39:11,F,1,10,48067
4,48,Pocahontas (1995),Children's,1,5,2001-01-06 23:39:11,F,1,10,48067


In [114]:
genre_ratings = (ratings_with_genre.loc[:, ["genres", "age", "rating"]]
                                   .groupby(["genres", "age"])
                                   .mean()
                                   .unstack("age"))

In [115]:
genre_ratings

rating                                                    \
age                1         18        25        35        45        50   
genres                                                                    
Action       3.506385  3.447097  3.453358  3.538107  3.528543  3.611333   
Adventure    3.449975  3.408525  3.443163  3.515291  3.528963  3.628163   
Animation    3.476113  3.624014  3.701228  3.740545  3.734856  3.780020   
Children's   3.241642  3.294257  3.426873  3.518423  3.527593  3.556555   
Comedy       3.497491  3.460417  3.490385  3.561984  3.591789  3.646868   
Crime        3.710170  3.668054  3.680321  3.733736  3.750661  3.810688   
Documentary  3.730769  3.865865  3.946690  3.953747  3.966521  3.908108   
Drama        3.794735  3.721930  3.726428  3.782512  3.784356  3.878415   
Fantasy      3.317647  3.353778  3.452484  3.482301  3.532468  3.581570   
Film-Noir    4.145455  3.997368  4.058725  4.064910  4.105376  4.175401   
Horror       3.254184  3.172682  3.200096  3.276022  3.262274  3.158940   
Musical      3.568306  3.560291  3.619857  3.721587  3.744484  3.798254   
Mystery      3.631522  3.525387  3.610818  3.697396  3.754350  3.885795   
Romance      3.621284  3.534300  3.546730  3.651142  3.685990  3.758111   
Sci-Fi       3.478698  3.426067  3.443822  3.502119  3.482515  3.564456   
Thriller     3.550373  3.494634  3.535471  3.615934  3.639657  3.709668   
War          3.895437  3.853136  3.841240  3.901130  3.960554  3.974184   
Western      3.576119  3.471533  3.607826  3.679278  3.667135  3.741322   

                       
age                56  
genres                 
Action       3.610709  
Adventure    3.649064  
Animation    3.756233  
Children's   3.621822  
Comedy       3.650949  
Crime        3.832549  
Documentary  3.961538  
Drama        3.933465  
Fantasy      3.532700  
Film-Noir    4.125932  
Horror       3.254401  
Musical      3.886713  
Mystery      3.890545  
Romance      3.816531  
Sci-Fi       3.497746  
Thriller     3.719749  
War          4.067285  
Western      3.792198

In [118]:
genre_ratings.columns

MultiIndex([('rating',  1),
            ('rating', 18),
            ('rating', 25),
            ('rating', 35),
            ('rating', 45),
            ('rating', 50),
            ('rating', 56)],
           names=[None, 'age'])

In [119]:
genre_ratings.sort_values(('rating',  1), ascending=False)

rating                                                    \
age                1         18        25        35        45        50   
genres                                                                    
Film-Noir    4.145455  3.997368  4.058725  4.064910  4.105376  4.175401   
War          3.895437  3.853136  3.841240  3.901130  3.960554  3.974184   
Drama        3.794735  3.721930  3.726428  3.782512  3.784356  3.878415   
Documentary  3.730769  3.865865  3.946690  3.953747  3.966521  3.908108   
Crime        3.710170  3.668054  3.680321  3.733736  3.750661  3.810688   
Mystery      3.631522  3.525387  3.610818  3.697396  3.754350  3.885795   
Romance      3.621284  3.534300  3.546730  3.651142  3.685990  3.758111   
Western      3.576119  3.471533  3.607826  3.679278  3.667135  3.741322   
Musical      3.568306  3.560291  3.619857  3.721587  3.744484  3.798254   
Thriller     3.550373  3.494634  3.535471  3.615934  3.639657  3.709668   
Action       3.506385  3.447097  3.453358  3.538107  3.528543  3.611333   
Comedy       3.497491  3.460417  3.490385  3.561984  3.591789  3.646868   
Sci-Fi       3.478698  3.426067  3.443822  3.502119  3.482515  3.564456   
Animation    3.476113  3.624014  3.701228  3.740545  3.734856  3.780020   
Adventure    3.449975  3.408525  3.443163  3.515291  3.528963  3.628163   
Fantasy      3.317647  3.353778  3.452484  3.482301  3.532468  3.581570   
Horror       3.254184  3.172682  3.200096  3.276022  3.262274  3.158940   
Children's   3.241642  3.294257  3.426873  3.518423  3.527593  3.556555   

                       
age                56  
genres                 
Film-Noir    4.125932  
War          4.067285  
Drama        3.933465  
Documentary  3.961538  
Crime        3.832549  
Mystery      3.890545  
Romance      3.816531  
Western      3.792198  
Musical      3.886713  
Thriller     3.719749  
Action       3.610709  
Comedy       3.650949  
Sci-Fi       3.497746  
Animation    3.756233  
Adventure    3.649064  
Fantasy      3.532700  
Horror       3.254401  
Children's   3.621822